<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import wget
import pandas as pd
print('all libaries imported.')

all libaries imported.


In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
print('all packages imported.')

all packages imported.


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 8966 / 8966

In [4]:
spacex_df.head(2)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

for lat, lng, label in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df['Launch Site']):
    circle = folium.Circle([lat, lng], radius = 100, color = 'coral', fill = True).add_child(folium.Popup(label))
    marker = folium.map.Marker([lat, lng],
                              icon = DivIcon(
                              icon_size = (40, 10),
                              icon_anchor = (0,0),
                              html = '<div style="font-size:8; color:coral;"><b>%s</b></div>' % label))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x:'green' if x == 1 else 'red')
spacex_df.head(2)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
site_map.add_child(marker_cluster)

# Add marker_cluster to current site_map
for index, record in spacex_df.iterrows():
    marker = folium.Marker([record[1], record[2]],
                          icon = folium.Icon(color = 'white', icon_color = record[4]))
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [82]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [48]:
# Launch Site CCAFS LC-40
# Lat: 28.562302
# Lng: -80.577356
lat_ls1 = 28.562302
lng_ls1 = -80.577356
lat_cl1 = 28.56296
lng_cl1 = -80.56798
distance_ls1_cl1 = calculate_distance(lat_ls1, lng_ls1, lat_cl1, lng_cl1)
lat_hw1 = 28.56484
lng_hw1 = -80.57101
distance_hw1 = calculate_distance(lat_ls1, lng_ls1, lat_hw1, lng_hw1)
lat_rw1 = 28.5719
lng_rw1 = -80.58532
distance_rw1 = calculate_distance(lat_ls1, lng_ls1, lat_rw1, lng_rw1)

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [50]:
site_map = folium.Map([lat_ls1, lng_ls1], zoom_start = 15)
circle_1 = folium.Circle([lat_ls1, lng_ls1], radius = 50, color = 'seagreen', fill = True).add_child(folium.Popup('CCAFS LC-40')).add_to(site_map)
circle_cl1 = folium.Circle([lat_cl1, lng_cl1], radius = 50, color = 'blue', fill = True).add_to(site_map)
circle_hw1 = folium.Circle([lat_hw1, lng_hw1], radius = 50, color = 'grey', fill = True).add_to(site_map)
circle_rw1 = folium.Circle([lat_rw1, lng_rw1], radius = 50, color = 'brown', fill = True).add_to(site_map)
line_cl1 = folium.PolyLine([[lat_ls1, lng_ls1], [lat_cl1, lng_cl1]], weight = 2, color = 'blue').add_to(site_map)
line_hw1 = folium.PolyLine([[lat_ls1, lng_ls1], [lat_hw1, lng_hw1]], weight = 2, color = 'grey').add_to(site_map)
line_rw1 = folium.PolyLine([[lat_ls1, lng_ls1], [lat_rw1, lng_rw1]], weight = 2, color = 'brown').add_to(site_map)
distance_marker_cl1 = folium.Marker([((lat_ls1 + lat_cl1)/2), ((lng_ls1 + lng_cl1)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (0, 0),
                               html = '<div style="font-size: 6; color:blue;"><b>%s</b></div>' % "{:10.2f} KM to coast line".format(
                               distance_ls1_cl1))).add_to(site_map)
distance_marker_hw1 = folium.Marker([((lat_ls1 + lat_hw1)/2), ((lng_ls1 + lng_hw1)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (0, 0),
                               html = '<div style="font-size: 6; color:grey;"><b>%s</b></div>' % "{:10.2f} KM to highway".format(
                               distance_hw1))).add_to(site_map)
distance_marker_hw1 = folium.Marker([((lat_ls1 + lat_rw1)/2), ((lng_ls1 + lng_rw1)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (0, 0),
                               html = '<div style="font-size: 6; color:brown;"><b>%s</b></div>' % "{:10.2f} KM to railway".format(
                               distance_rw1))).add_to(site_map)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [52]:
# Launch Site KSC LC-39A
# Lat: 28.573255
# Lng: -80.646895
lat_ls2 = 28.573255
lng_ls2 = -80.646895
lat_cl2 = 28.57383
lng_cl2 = -80.60682
distance_ls2_cl2 = calculate_distance(lat_ls2, lng_ls2, lat_cl2, lng_cl2)
lat_hw2 = 28.57341
lng_hw2 = -80.65541
distance_hw2 = calculate_distance(lat_ls2, lng_ls2, lat_hw2, lng_hw2)
lat_rw2 = 28.57373
lng_rw2 = -80.65417
distance_rw2 = calculate_distance(lat_ls2, lng_ls2, lat_rw2, lng_rw2)

In [68]:
site_map = folium.Map([lat_ls2, lng_ls2], zoom_start = 14)
circle_2 = folium.Circle([lat_ls2, lng_ls2], radius = 50, color = 'seagreen', fill = True).add_child(folium.Popup('KSC LC-39A')).add_to(site_map)
circle_cl2 = folium.Circle([lat_cl2, lng_cl2], radius = 50, color = 'blue', fill = True).add_to(site_map)
circle_hw2 = folium.Circle([lat_hw2, lng_hw2], radius = 50, color = 'grey', fill = True).add_to(site_map)
circle_rw2 = folium.Circle([lat_rw2, lng_rw2], radius = 50, color = 'brown', fill = True).add_to(site_map)
line_cl2 = folium.PolyLine([[lat_ls2, lng_ls2], [lat_cl2, lng_cl2]], weight = 2, color = 'blue').add_to(site_map)
line_hw2 = folium.PolyLine([[lat_ls2, lng_ls2], [lat_hw2, lng_hw2]], weight = 2, color = 'grey').add_to(site_map)
line_rw2 = folium.PolyLine([[lat_ls2, lng_ls2], [lat_rw2, lng_rw2]], weight = 2, color = 'brown').add_to(site_map)
distance_marker_cl2 = folium.Marker([((lat_ls2 + lat_cl2)/2), ((lng_ls2 + lng_cl2)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (0, 0),
                               html = '<div style="font-size: 6; color:blue;"><b>%s</b></div>' % "{:10.2f} KM to coast line".format(
                               distance_ls2_cl2))).add_to(site_map)
distance_marker_hw2 = folium.Marker([((lat_ls2 + lat_hw2)/2), ((lng_ls2 + lng_hw2)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (60, 0),
                               html = '<div style="font-size: 6; color:grey;"><b>%s</b></div>' % "{:10.2f} KM to highway".format(
                               distance_hw2))).add_to(site_map)
distance_marker_hw2 = folium.Marker([((lat_ls2 + lat_rw2)/2), ((lng_ls2 + lng_rw2)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (40, 30),
                               html = '<div style="font-size: 6; color:brown;"><b>%s</b></div>' % "{:10.2f} KM to railway".format(
                               distance_rw2))).add_to(site_map)
site_map

In [73]:
# VAFB SLC-4E
# Lat: 34.632834
# Lng: -120.610746
lat_ls3 = 34.632834
lng_ls3 = -120.610746
lat_cl3 = 34.63474
lng_cl3 = -120.62514
distance_cl3 = calculate_distance(lat_ls3, lng_ls3, lat_cl3, lng_cl3)
lat_hw3 = 34.6388
lng_hw3 = -120.62029
distance_hw3 = calculate_distance(lat_ls3, lng_ls3, lat_hw3, lng_hw3)
lat_rw3 = 34.63604
lng_rw3 = -120.6239
distance_rw3 = calculate_distance(lat_ls3, lng_ls3, lat_rw3, lng_rw3)

In [75]:
site_map = folium.Map([lat_ls3, lng_ls3], zoom_start = 15)
circle_3 = folium.Circle([lat_ls3, lng_ls3], radius = 50, color = 'seagreen', fill = True).add_child(folium.Popup('VAFB SLC-4E')).add_to(site_map)
circle_cl3 = folium.Circle([lat_cl3, lng_cl3], radius = 50, color = 'blue', fill = True).add_to(site_map)
circle_hw3 = folium.Circle([lat_hw3, lng_hw3], radius = 50, color = 'grey', fill = True).add_to(site_map)
circle_rw3 = folium.Circle([lat_rw3, lng_rw3], radius = 50, color = 'brown', fill = True).add_to(site_map)
line_cl3 = folium.PolyLine([[lat_ls3, lng_ls3], [lat_cl3, lng_cl3]], weight = 2, color = 'blue').add_to(site_map)
line_hw3 = folium.PolyLine([[lat_ls3, lng_ls3], [lat_hw3, lng_hw3]], weight = 2, color = 'grey').add_to(site_map)
line_rw3 = folium.PolyLine([[lat_ls3, lng_ls3], [lat_rw3, lng_rw3]], weight = 2, color = 'brown').add_to(site_map)
distance_marker_cl3 = folium.Marker([((lat_ls3 + lat_cl3)/2), ((lng_ls3 + lng_cl3)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (0, 0),
                               html = '<div style="font-size: 6; color:blue;"><b>%s</b></div>' % "{:10.2f} KM to coast line".format(
                               distance_cl3))).add_to(site_map)
distance_marker_hw3 = folium.Marker([((lat_ls3 + lat_hw3)/2), ((lng_ls3 + lng_hw3)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (60, 0),
                               html = '<div style="font-size: 6; color:grey;"><b>%s</b></div>' % "{:10.2f} KM to highway".format(
                               distance_hw3))).add_to(site_map)
distance_marker_hw3 = folium.Marker([((lat_ls3 + lat_rw3)/2), ((lng_ls3 + lng_rw3)/2)],
                               icon = DivIcon(
                               icon_size = (200, 40),
                               icon_anchor = (40, 30),
                               html = '<div style="font-size: 6; color:brown;"><b>%s</b></div>' % "{:10.2f} KM to railway".format(
                               distance_rw3))).add_to(site_map)
site_map

In [76]:
import geopy 
from geopy import Nominatim

locator = Nominatim(user_agent = 'my_geocoder')
location = locator.geocode('Melbourne, USA')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 28.106471, Longitude = -80.6371513


In [95]:
lat_mel = 28.106471
lng_mel = -80.6371513

distance_mel = calculate_distance(lat_mel, lng_mel, lat_ls1, lng_ls1)
distance_mel1 = calculate_distance(lat_mel, lng_mel, lat_ls2, lng_ls2)

site_map = folium.Map([((lat_ls1 + lat_mel)/2), ((lng_ls1 + lng_mel)/2)], zoom_start = 10)
folium.Circle([lat_mel, lng_mel], radius = 500, color = 'black', fill = True).add_to(site_map)
folium.Circle([lat_ls1, lng_ls1], radius = 500, color = 'seagreen', fill = True).add_to(site_map)
folium.Circle([lat_ls2, lng_ls2], radius = 500, color = 'seagreen', fill = True).add_to(site_map)
folium.Marker([((lat_ls1 + lat_mel)/2), ((lng_ls1 + lng_mel)/2)],
                               icon = DivIcon(
                               icon_size = (400, 40),
                               icon_anchor = (0, 20),
                               html = '<div style="font-size: 6; color:black;"><b>%s</b></div>' % "CCAFS LC-40 {:10.2f} KM to Melbourne".format(distance_mel))).add_to(site_map)
folium.Marker([((lat_ls2 + lat_mel)/2), ((lng_ls2 + lng_mel)/2)],
                               icon = DivIcon(
                               icon_size = (400, 40),
                               icon_anchor = (0, -20),
                               html = '<div style="font-size: 6; color:black;"><b>%s</b></div>' % "KSC LC-39A {:10.2f} KM to Melbourne".format(distance_mel1))).add_to(site_map)
folium.PolyLine([[lat_mel, lng_mel],[lat_ls1, lng_ls1]], weight = 2, color = 'black').add_to(site_map)
folium.PolyLine([[lat_mel, lng_mel],[lat_ls2, lng_ls2]], weight = 2, color = 'black').add_to(site_map)
site_map

In [94]:
# VAFB SLC-4E
lat_ls3 = 34.632834
lng_ls3 = -120.610746

lat_san = locator.geocode('Santa Maria, USA').latitude
lng_san = locator.geocode('Santa Maria, USA').longitude
distance_san = calculate_distance(lat_ls3, lng_ls3, lat_san, lng_san)

In [99]:
site_map = folium.Map([((lat_ls3 + lat_san)/2), ((lng_ls3 + lng_san)/2)], zoom_start = 10)
folium.Circle([lat_ls3, lng_ls3], radius = 500, color = 'seagreen', fill = True). add_to(site_map)
folium.Circle([lat_san, lng_san], radius = 500, color = 'black', fill = True).add_to(site_map)
folium.Marker([((lat_ls3 + lat_san)/2), ((lng_ls3 + lng_san)/2)], 
             icon = DivIcon(
             icon_size = (400, 40),
             icon_anchor = (0, 0),
             html = '<div style="font-size:6; color:black;"><b>%s</b></div>' % "VAFB SLC-4E {:10.2F} KM to Santa Maria".format(
             distance_san))).add_to(site_map)
folium.PolyLine([[lat_ls3, lng_ls3], [lat_san, lng_san]], weight = 2, color = 'black').add_to(site_map)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
